<a href="https://colab.research.google.com/github/TechTM-0/mediapipe/blob/main/Mediapipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

マウントしてColabからGoogleドライブにアクセスできるように設定

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


mediapipeインストール

In [1]:
!pip install mediapipe

インストール確認

In [4]:
import mediapipe as mp

try:
    print("MediaPipeのバージョン:", mp.__version__)
    print("MediaPipeは正常に動作しています！")
except AttributeError:
    print("MediaPipeはインポートできましたが、バージョン情報を取得できませんでした。")
except Exception as e:
    print(f"エラーが発生しました: {e}")

MediaPipeのバージョン: 0.10.21
MediaPipeは正常に動作しています！


In [10]:
import cv2
import mediapipe as mp

# === 入力・出力ファイルの設定 ===
# 処理する動画のGoogleドライブ上のパス
# ご自身のファイルパスに合わせて変更してください
input_video_path = '/content/drive/MyDrive/material/140642-775595913_small.mp4'

# 処理結果を保存する動画のGoogleドライブ上のパス
# ファイル名は自由に変更できます
output_video_path = '/content/drive/MyDrive/output/output_video_skeleton_2.mp4'

# === MediaPipe Pose（骨格検出）の準備 ===
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False)

# === 動画の読み込みと設定の取得 ===
# 入力動画を読み込む
cap = cv2.VideoCapture(input_video_path)

# 動画が正常に開いたか確認
if not cap.isOpened():
    print(f"エラー: 入力動画 '{input_video_path}' を開けませんでした。")
    exit()

# 入力動画の情報を取得
# 画面サイズ (幅, 高さ) を取得
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# フレームレート（1秒あたりのフレーム数）を取得
fps = int(cap.get(cv2.CAP_PROP_FPS))

# === 出力動画の設定 ===
# 動画ファイルのコーデックを指定 (MP4形式で保存するための推奨設定)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# 動画を書き込むためのVideoWriterオブジェクトを初期化
writer = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

print("動画処理を開始します...")

# === メインの処理ループ ===
# 動画の全フレームを1つずつ処理
while cap.isOpened():
    # フレームを読み込む
    ret, frame = cap.read()
    if not ret:
        # フレームが読み込めなくなったらループを終了
        break

    # MediaPipeが処理しやすいように色空間をBGRからRGBに変換
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 骨格情報を検出
    results = pose.process(image_rgb)

    # 検出された骨格情報を元のフレームに描画
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            frame,  # 描画対象のフレーム
            results.pose_landmarks,  # 検出されたランドマーク情報
            mp_pose.POSE_CONNECTIONS,  # 骨格の接続関係
            # ランドマークと線の色や太さをカスタマイズ
            # 1つ目(ランドマーク),2つ目(接続線)
            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=1, circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)
        )

    # 骨格を描画したフレームを出力動画ファイルに書き込む
    writer.write(frame)

# === 後処理 ===
# 処理が完了したら、全ての動画リソースを解放する
cap.release()
writer.release()
pose.close()

print("動画処理が完了しました。")
print(f"'{output_video_path}' に動画が保存されました。")

動画処理を開始します...
動画処理が完了しました。
'/content/drive/MyDrive/output/output_video_skeleton_2.mp4' に動画が保存されました。


In [14]:
import cv2
import mediapipe as mp

# === 入力・出力ファイルの設定 ===
# 処理する動画のGoogleドライブ上のパス
input_video_path = '/content/drive/MyDrive/material/PXL_20250830_171433857.TS.mp4'

# 処理結果を保存する動画のGoogleドライブ上のパス
output_video_path = '/content/drive/MyDrive/output/output_video_hand_pose.mp4'

# === MediaPipe Pose（骨格検出）とHands（手の検出）の準備 ===
mp_pose = mp.solutions.pose
# 手の検出モジュールを追加
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# ポーズ検出モデルを初期化
pose = mp_pose.Pose(static_image_mode=False)
# 手の検出モデルを初期化
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,  # 最大2つの手を検出
    min_detection_confidence=0.5
)

# === 動画の読み込みと設定の取得 ===
cap = cv2.VideoCapture(input_video_path)

if not cap.isOpened():
    print(f"エラー: 入力動画 '{input_video_path}' を開けませんでした。")
    exit()

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# === 出力動画の設定 ===
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

print("動画処理を開始します...")

# === メインの処理ループ ===
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # ポーズ（骨格）情報を検出
    pose_results = pose.process(image_rgb)
    # 手の情報を検出
    hands_results = hands.process(image_rgb)

    検出されたポーズ情報を元のフレームに描画
    if pose_results.pose_landmarks:
        mp_drawing.draw_landmarks(
            frame,
            pose_results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=1, circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)
        )

    # 検出された手の情報を元のフレームに描画
    if hands_results.multi_hand_landmarks:
        for hand_landmarks in hands_results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=2, circle_radius=2)
            )

    writer.write(frame)

# === 後処理 ===
cap.release()
writer.release()
pose.close()
# handsオブジェクトをクリーンアップ
hands.close()

print("動画処理が完了しました。")
print(f"'{output_video_path}' に動画が保存されました。")

動画処理を開始します...
動画処理が完了しました。
'/content/drive/MyDrive/output/output_video_hand_pose.mp4' に動画が保存されました。


手の検出精度を上げる

In [22]:
import cv2
import mediapipe as mp
import numpy as np

# === 入力・出力ファイルの設定 ===
# 処理する動画のGoogleドライブ上のパス。このパスはユーザーが変更する必要があります。
input_video_path = '/content/drive/MyDrive/material/140642-775595913_small.mp4'

# 処理結果を保存する新しい動画ファイルのパス。
output_video_path = '/content/drive/MyDrive/output/output_video_hand_pose_cropped_3.mp4'

# === MediaPipe PoseとHandsの準備 ===
# ポーズ（骨格）検出用のライブラリをインポート。
mp_pose = mp.solutions.pose
# 手のランドマーク検出用のライブラリをインポート。
mp_hands = mp.solutions.hands
# 検出結果を描画するためのユーティリティをインポート。
mp_drawing = mp.solutions.drawing_utils

# ポーズ（骨格）検出モデルを初期化。
# static_image_mode=Falseに設定することで、動画（連続したフレーム）の処理に適したモードになります。
pose = mp_pose.Pose(static_image_mode=False)

# 手の検出モデルを初期化。
# static_image_mode=False：動画（連続したフレーム）モードで動作します。
# max_num_hands=2：同時に検出する手の数を最大2つに設定します。
# min_detection_confidence=0.3：検出の信頼度をデフォルトの0.5から0.3に下げることで、
# 小さく映っている手など、見逃しがちな手も捉えやすくなります。
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.3
)

# === 動画の読み込みと設定の取得 ===
# OpenCVを使って入力動画ファイルを開きます。
cap = cv2.VideoCapture(input_video_path)

# 動画ファイルが正常に開かれたかを確認します。
if not cap.isOpened():
    print(f"エラー: 入力動画 '{input_video_path}' を開けませんでした。")
    exit()

# 動画の幅、高さ、フレームレート（fps）を取得します。
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# === 出力動画の設定 ===
# 出力動画のコーデックを定義します。'mp4v'はMP4形式で広く使われます。
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# 出力動画ファイルを作成します。元の動画と同じ設定（幅、高さ、fps）にします。
writer = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

print("動画処理を開始します...")

# === メインループ：動画の全フレームを処理 ===
# 動画が開いている間、無限ループでフレームを1枚ずつ読み込みます。
while cap.isOpened():
    # フレームを1枚読み込みます。
    ret, frame = cap.read()
    # フレームが読み込めなかった場合（動画の終端に達した場合など）はループを終了します。
    if not ret:
        break

    # MediaPipeの処理のために、色空間をBGRからRGBに変換します。
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 最初の処理として、フレーム全体でポーズ（骨格）を検出します。
    # これにより、後で手の位置を特定するための情報（手首の座標）が得られます。
    pose_results = pose.process(image_rgb)

    # ポーズが検出された場合のみ、以下の処理を実行します。
    if pose_results.pose_landmarks:
        # ポーズ（骨格）の描画設定を定義します。
        # 線の色をオレンジ、点のサイズを小さくします。
        pose_spec = mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=1, circle_radius=2)
        # 点の色を緑、サイズを少し大きくします。
        pose_dot_spec = mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)

        # 検出されたポーズ（骨格）のランドマークと接続線を元のフレームに描画します。
        mp_drawing.draw_landmarks(
            frame,
            pose_results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            pose_spec,
            pose_dot_spec
        )

        # 左右の手首のランドマークをポーズの検出結果から取得します。
        left_wrist = pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]
        right_wrist = pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST]

        # 左右の手首それぞれに対してループを回し、個別に手の検出と描画を行います。
        for wrist_landmark in [left_wrist, right_wrist]:
            # 手首のランドマークの信頼度が0.5より高い(手首が正しく検出できている)場合のみ処理を続行します。
            # これにより、誤検出による無駄な処理を防ぎます。
            if wrist_landmark.visibility > 0.3:
                # ランドマークの正規化された座標（0.0～1.0）です。
                # フレームの長さ(ピクセル)を掛けることで実際のピクセル座標に変換します。

                center_x = int(wrist_landmark.x * frame_width)
                center_y = int(wrist_landmark.y * frame_height)

                # 手を検出するために切り抜く領域のサイズを定義します。
                size = 200
                # 切り抜き領域の左上と右下の座標を計算します。
                # max()とmin()を使って、領域がフレームの範囲外に出ないように調整します。
                x1 = max(0, center_x - size // 2)
                y1 = max(0, center_y - size // 2)
                x2 = min(frame_width, center_x + size // 2)
                y2 = min(frame_height, center_y + size // 2)

                # 計算した座標に基づいて、元のフレームから手の領域を切り抜きます（クロッピング）。
                hand_crop = frame[y1:y2, x1:x2]

                # 切り抜いた領域が有効（サイズが0より大きい）な場合にのみ、次の処理に進みます。
                if hand_crop.size > 0:
                    # 切り抜いた画像をRGBに変換し、手のランドマークを再検出します。
                    # 小さく映っていた手が拡大されてモデルに渡されるため、検出精度が向上します。
                    hand_crop_rgb = cv2.cvtColor(hand_crop, cv2.COLOR_BGR2RGB)
                    hands_results = hands.process(hand_crop_rgb)

                    # 手が検出された場合のみ描画を行います。
                    if hands_results.multi_hand_landmarks:
                        for hand_landmarks in hands_results.multi_hand_landmarks:
                            # 手のランドマークの描画設定を定義します。
                            # 線の色を赤、点のサイズを小さくします。
                            hand_spec = mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=1, circle_radius=1)
                            # 点の色をシアン、サイズを小さくします。
                            hand_dot_spec = mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)

                            # 切り抜いた画像に、手のランドマークと接続線を描画します。
                            mp_drawing.draw_landmarks(
                                hand_crop,
                                hand_landmarks,
                                mp_hands.HAND_CONNECTIONS,
                                hand_spec,
                                hand_dot_spec
                            )

                            # ランドマークが描画された切り抜き画像を、元のフレームの正しい位置に貼り付け直します。
                            frame[y1:y2, x1:x2] = hand_crop

    # 処理済みのフレームを出力動画ファイルに書き込みます。
    writer.write(frame)

# === 後処理 ===
# 動画ファイルオブジェクトとモデルを解放し、リソースをクリーンアップします。
# これを怠ると、出力動画が破損することがあります。
cap.release()
writer.release()
pose.close()
hands.close()

print("動画処理が完了しました。")
print(f"'{output_video_path}' に動画が保存されました。")

動画処理を開始します...
動画処理が完了しました。
'/content/drive/MyDrive/output/output_video_hand_pose_cropped_3.mp4' に動画が保存されました。
